# MongoDB

From Introduction to MongoDB at https://docs.mongodb.com/manual/introduction/:  
"MongoDB is an open-source document database that provides high performance, high availability, and automatic scaling."

While many relational database management systems have been in use since the 1970s, 
as technology and how we use it continues to evolve we need new ways to address the growing volume of data, and how we interact with it. 

What is meant by a "document database" is that each entry in the database is meant to be a self-contained unit. 
These so called documents are then organized into what are called "collections".


While joins are possible in MongoDB, 
they are a bit of an "anti-pattern" in the context of a document store. 



**At the very basic level,
a document is similar to a row in a relational database, 
and a collection is analogous to a table.** 


## Node Assignments 

* Server `mongodb-1.dsa.lan`: Last name A - G 
* Server `mongodb-2.dsa.lan`: Last name H - M
* Server `mongodb-3.dsa.lan`: Last name N - S
* Server `mongodb-4.dsa.lan`: Last name T - Y


## Readings
* [Why NoSQL Database](https://www.couchbase.com/resources/why-nosql)
* [What is MongoDB](https://intellipaat.com/blog/what-is-mongodb/)
* [Introduction to MongoDB and Python](https://realpython.com/introduction-to-mongodb-and-python/)

## Intro to MongoDB + Python with a movie review dataset

Let's start exploring MongoDB with Python to see how things compare to PostgreSQL. 

Full documentation on `pymongo` can be found here:https://api.mongodb.com/python/current/api/pymongo/

In [ ]:
from pymongo import MongoClient
import pymongo

# Initialize a Mongo Client
#################################################
# Update UPDATE-ME in the connection code
#################################################
# Node 1 - mongodb-1.dsa.lan
# Node 2 - mongodb-2.dsa.lan
# Node 3 - mongodb-3.dsa.lan
# Node 4 - mongodb-4.dsa.lan
#################################################
#
client = MongoClient('UPDATE-ME.dsa.lan',
                     username='ml_small_reader',
                     password='mlsmall.read',
                     authSource='ml_small')

In [ ]:
# connect to the ml_small database from the connection
db = client.ml_small

In [ ]:
# Get a list of the available collections in this DB.
db.list_collection_names()

In [ ]:
# Get a count of the number of documents in the item collection
print(f"num of items = {db.item.count_documents({})}")
print(f"num of users = {db.user.count_documents({})}")
print(f"num of genres = {db.genre.count_documents({})}")
print(f"num of user ratings = {db.data.count_documents({})}")

**Note**: in the function call above, we passed in a Dictionary, that mongo converts to a JSON document, that is empty.
This _document_ is the query parameters, i.e., how to match documents it is to count.

With some basic syntax examples, let's explore the data.

In [ ]:
db.item.find_one()

In [ ]:
db.user.find_one()

In [ ]:
db.genre.find_one()

In [ ]:
db.data.find_one()

Now that we've seen our data, let's make sense of it. 

This data is a movie rating dataset for 1681 movies [`db.item.count_documents({})`]. 
There are 943 users [`db.user.count_documents({})`] who have submitted, 
in this dataset, 100k reviews [`db.data.count_documents({})`]. 

`db.genre` is a listing of all of the genre's available, along with their unique ID, 
which is not used elsewhere in this dataset.
This is residual data from the fact that this dataset was exported from a relational schema.

Each document in `db.item` describes a single movie. 
Each movie could fit into any number of genres, 
so all genres are listed and a binary yes (1) or no (0) is recorded if the movie fit that genre. 
We also have some other data available, including the movie title. 

We'll dive into the data more with the Aggregates and Joins labs, 
but let's get some examples on how to do things you would naturally think of trying to do.

## Returning a number of rows

In [ ]:
# get an iterator for all genres.
for genre in db.genre.find():
    print(genre)
    

In [ ]:
# Or get the iterator and immediately make it a list...
genres = list(db.genre.find())
print(genres)
print('-'*50)
print(len(genres))


In [ ]:
# Find a specific Genre, knowing the query will always return at-most one document.
print(db.genre.find_one({"genre": "Children's"}))
print(db.genre.find_one({"genre": "Sci-Fi"}))

In [ ]:
# Find any single document, with no concern for which.
print(db.genre.find_one())

## Sorting

In [ ]:
# Find the last genre, in alphabetical order.
print(db.genre.find_one(sort=[('genre', pymongo.DESCENDING)]))

# Likewise the first one, in alphabetical order.
print(db.genre.find_one(sort=[('genre', pymongo.ASCENDING)]))

## Sorting with a limit 

In [ ]:
for movie in db.item.find(sort=[('movie_title', pymongo.ASCENDING)], limit=20):
    print(movie['movie_title'])

## Sorting with a limit and skipping documents

In [ ]:
# Skip the movies whose title begins with a number 
for movie in db.item.find(sort=[('movie_title', pymongo.ASCENDING)], 
                          limit=20, skip=13):
    print(movie['movie_title'])

## Summary

As you can see, 
while the syntax for interacting with MongoDB is a little different from what you might 
expect if you are used to working with SQL statements, 
the interface can be picked up relatively quickly. 
Additionally, most things that you would want to do in SQL have a mapping to NoSQL. 

We skip over the data modification statements in this lab since we are connected to a read-only database, 
however they are much the same as you would expect: 
 * `db.collection.insert_*()`, 
 * `db.collection.update_*()`, 
 * `db.collection.delete_*()`,
 
are all available, as well as some not-so-obvious DML such as `db.collection.find_one_and_*()` functions. 

In the prior examples, `*` is a placeholder for one/many and then insert/update/delete, depending on the relevant context. 
See the documentation for more specifics. 


## <span style="background:yellow">Your Turn</span>

Complete following lab work to gain hands-on experience with PyMongo and MongoDB.
**Make sure that you have ran the above cells, and have a connection to the DB!**

### Task 1

#### Query the database to get the movies that start with the letter "a", case-insensitive. 
Specifically, the regex pattern is `'^a'` with option `'i'`
 * refer to https://docs.mongodb.com/manual/reference/operator/query/regex/ for help.

In [ ]:
# Add your code below
# -------------------------






### Task 2

#### Query to find movies that were released after Janurary 1, 1995. 

As an example, and to aid you, datetime.datetime has been imported, 
the afterDate variable has been created for use in your query. 

Hint: It is Janurary 2, so  use greater than or equal to.
 * https://docs.mongodb.com/manual/reference/method/db.collection.find/ 

In [ ]:
from datetime import datetime
afterDate = datetime(1995, 1, 2)

# Add your code below
# -------------------------







---

In [ ]:
# Be sure to run this cell when you are finished. Thank you.
client.close()

# Save your notebook, then `File > Close and Halt`

---